**PAQUETERIAS NECESARIAS**

In [1]:
## INCLUYE TODOS LOS PAQUETES A USAR
import tensorflow as tf
import random
from tensorflow import keras
 # Bibliotecas de ayuda
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import gaussian
 # Biblioteca para datos de entrenamiento
from sklearn.model_selection import train_test_split
import h5py as h # manipular extensiones
# FUNCIONES DEFINIDAS PARA USO NECESARIO
import timeit

**ENTRADA DE LA INFORMACION, ORDENAMIENTO DE LA INFORMACION A USAR**

In [2]:
 # ENTRAR LA INFORMACION DESDE UN ARCHIVO EXTERNO
FILE  = h.File("DATOS/TODOS.h5", 'r')
INPUT = np.vstack([FILE["qcd/images"][:], FILE["wprime/images"][:]]); INPUT=INPUT/np.max(INPUT) # SE NORMALIZA
LABEL = np.hstack([FILE["qcd/labels"][:], FILE["wprime/labels"][:]])
OUTPUT_LINEAL = np.ones((len(LABEL), 2));
OUTPUT_LINEAL[:,0] = LABEL < 1    ;     OUTPUT_LINEAL[:,1] = LABEL > 0
 # REACONDICIONAR LA INFORMACION INPUT EN UNA MATRIX PARA SU RAPIDO ACCESO
INPUT_LINEAL = np.reshape(INPUT, [len(INPUT),-1]) # LA INFORMACION SE LINEALIZA Y SE LOCALIZA EN LAS FILAS
 # SEPARAR LA INFO PARA ENTRENAR DE LA QUE SERA USADA PARA EL TEST DE CORRESPONDENCIA
INPUT_LINEAL, INPUT_LINEAL_TEST, OUTPUT_LINEAL, OUTPUT_LINEAL_TEST = train_test_split( 
    INPUT_LINEAL, OUTPUT_LINEAL, test_size=0.2, shuffle = True);

**FUNCIONES INICIALES Y NECESARIAS PARA EL ENTRENAMIENTO**

In [3]:
## CLASE DE LOS VALORES A TENER EN CUENTA
config = tf.compat.v1.placeholder("float")
config.v_input= len(INPUT_LINEAL[0,:]); config.v_output = len(OUTPUT_LINEAL[0,:]);#, exp = 1 , cross_entropy = 1, n_iterations = 3000, dropout = 0.5):
config.n_capas  = 2;                    config.exp      = 1;
#self.matrix_neuronas = {} # SE RELLENARA EN EL FUTURO
config.learning_rate = 1e-1; config.cross_entropy = 1;    config.n_iterations  = 3000;
config.dropout       = 0.5;
config.n_step_visual_process = 10;
config.acc_corte = .8;
## NO DEBEN VOLVERSE A DEFINIR BAJO NINGUN CONCEPTO
config.X = tf.compat.v1.placeholder("float", shape = [None,len(INPUT_LINEAL[0,:])] ) #, name= "input_size2"); # Estructura de la entrada
config.Y = tf.compat.v1.placeholder("float", shape = [None,len(OUTPUT_LINEAL[0,:])] ) #, name= "output_size2") #Estructura de la salida
    

In [4]:
## DEFINICION DE LOS MODOS DE ELECCION DE CAPAS OCULTAS Y NUMEROS DE NEURONAS
def neuronas( config ): #v_input, v_output, n_capas = 2, exp = 1,  modo = 'poly' ):
    #config = Dd()
    x = np.ones(config.n_capas + 2)
    #print(x)
    if  config.exp >= 1: #modo == 'poly':
        for i in range(config.n_capas + 2):
            x[len(x)-i-1] = round(( config.v_input - config.v_output )*pow(i/(config.n_capas + 1 ), config.exp) + config.v_output)
            #print(x[i])
            #print(i)
    elif config.exp < 1:
        for i in range(config.n_capas + 2):
            x[len(x)-i-1] = round(( config.v_input - config.v_output )*pow(i/( config.n_capas + 1 ), -1/config.exp) + config.v_output)
            #print(x[i])
            #print(i)
    config.matrix_neuronas = np.int64(x)
    return config

In [5]:
## ORGANIZACION DE LA ESTRUCTURA INTERNA DE ENTRENAMIENTO ##
def estructura( INPUT_LINEAL, OUTPUT_LINEAL , config):
    config = neuronas( config )
    #proceso iteractivo dinamico 
    for i in range(len(config.matrix_neuronas)-1):

        weights = tf.Variable( tf.truncated_normal([config.matrix_neuronas[i], config.matrix_neuronas[i+1]], stddev=0.1) , 
                              name = f"weights-w{i}")
        biases = tf.Variable(tf.constant(0.1, shape=[config.matrix_neuronas[i+1]]) , name=f"biases_b{i}" )
        
        if  i == 0: # CONDICION INICIAL
            layer = tf.add(tf.matmul( config.X, weights), biases , name=f"layer_l{i}" )
        else: # VALORES INTERNMEDIOS
            layer = tf.add(tf.matmul(    layer, weights), biases , name=f"layer_l{i}" )
    # PARAMETROS FINALES PARA CARACTERIZACION 
    config.cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = config.Y, logits=layer) , 
                                          name= "Definition_of_cross_entropy")
    config.train_step    = tf.compat.v1.train.AdamOptimizer( config.learning_rate ).minimize( config.cross_entropy , 
                                          name= "Definition_of_train_step")
    config.correct_pred  = tf.equal(tf.argmax(layer, 1), tf.argmax( config.Y , 1) , 
                                          name= "Definition_of_correct_pred")
    config.accuracy      = tf.reduce_mean(tf.cast( config.correct_pred  , tf.float32) ,
                                          name= "Definition_of_accuracy")
    return config

## INICIO DE LA ITERACION
def inicio( INPUT_LINEAL, INPUT_LINEAL_TEST, OUTPUT_LINEAL , OUTPUT_LINEAL_TEST , config):
    
    init = tf.compat.v1.global_variables_initializer()
    config.cost_summary = tf.compat.v1.summary.scalar("Cost", config.cross_entropy)
    config.acc_summary = tf.compat.v1.summary.scalar("Accuracy", config.accuracy)
    config.all_summary = tf.compat.v1.summary.merge_all()#'''
    #saver=tf.train.Saver() #SALVAR LA SECCION PARA PODERLA UTILIZAR LUEGO
    with tf.Session() as sess:
        writer = tf.summary.FileWriter("Tensorboard", sess.graph)
        #print(sess)
        sess.run(init)
        #saver.save(sess, 'modelo')
        toc_general = 0  
        for i in range(config.n_iterations):
            tic = timeit.default_timer()
            sess.run(config.train_step, feed_dict={ config.X: INPUT_LINEAL, config.Y:OUTPUT_LINEAL}) # CORRIDA

            config.summary_results, config.loss, config.acc = sess.run([config.all_summary, config.cross_entropy, config.accuracy], 
                                                  feed_dict={config.X: INPUT_LINEAL, config.Y: OUTPUT_LINEAL})
            writer.add_summary(config.summary_results, i)
            
            toc = timeit.default_timer() ;   toc_general +=  toc - tic ;
            if (i)%config.n_step_visual_process == 0:
                print("Iteration", str(i), "\t| Loss =", str( config.loss ), "\t| Accuracy =", str( config.acc )
                     , "\t| Time =", str( toc - tic ))
            
            if config.acc > config.acc_corte:
                print("Iteration", str(i), "\t| Loss =", str( config.loss ), "\t| Accuracy =", str( config.acc ))
                break
        config.test_accuracy = sess.run(config.accuracy, feed_dict={config.X: INPUT_LINEAL_TEST, config.Y: OUTPUT_LINEAL_TEST})
        config.time_ite_mean = toc_general/i ; 
        print("\n Accuracy on test set:", config.test_accuracy , "\t Mean time of Iterations:", str(config.time_ite_mean) )

        #a, b = sess.run([tf.argmax(layer, 1),tf.argmax(Y,1)], feed_dict={X: INPUT_LINEAL, Y: OUTPUT_LINEAL}) 

        #for ea, eb in zip(a, b):
        #    print(ea,eb)
    return config


**CARACTERIZACION DE NUESTRA MUESTRA Y NUESTRO PROCESO DE ENTRENAMIENTO**

In [6]:
## CARACTERIZAR COMO CAMBIA EL 

#config = neuronas( config )
config.learning_rate = .1 ; 
config = estructura( INPUT_LINEAL, OUTPUT_LINEAL , config )
config = inicio( INPUT_LINEAL, INPUT_LINEAL_TEST, OUTPUT_LINEAL , OUTPUT_LINEAL_TEST , config )

Iteration 0 	| Loss = 35.835873 	| Accuracy = 0.5974088 	| Time = 0.5117170000000009
Iteration 9 	| Loss = 1.916065 	| Accuracy = 0.83338094

 Accuracy on test set: 0.81142855 	 Mean time of Iterations: 0.4000330333333333
